# Manage models in production

This code example shows how to manage and monitor models deployed to production environments using DataRobot's Python client to accomplish various tasks: model deployment, replacement, deletion, and monitoring.

Download this notebook from the [code examples home page](index).

## Requirements

* Python version 3.7.3+
* DataRobot API version 2.19.0+
* A DataRobot `Project` object
* A DataRobot `Model` object

### Import libraries

In [2]:
import datarobot as dr

### Create a deployment

When creating a new deployment, you must provide a DataRobot model ID. The model ID represents a single instance of a model type, feature list, and sample size, used to differentiate models from the blueprint ID. The model ID is generated during Autopilot. Obtain the model ID from the UI by selecting the model to deploy from the Leaderboard and copying the string after `/models/` in the URL.

A prediction server makes predictions against a deployment and is required for each deployment. Use the default prediction server unless you are an Enterprise user, in which case you should use a preconfigured prediction server instead.

In [ ]:
# Get a list of prediction servers
prediction_server = dr.PredictionServer.list()[0]

project = dr.Project.get('<project_id>') # Provide your project object here
model = project.get_models()[0]

# Create a deployment
deployment = dr.Deployment.create_from_learning_model(
    model.id, label='New Deployment', description='A new deployment',
    default_prediction_server_id=prediction_server.id)
deployment

### List available deployments

Use the following command to list all available deployments.

In [ ]:
deployments = dr.Deployment.list()
deployments

You can filter the returned deployments by passing an instance of the `DeploymentListFilters` class to the `filters` keyword argument:

In [ ]:
filters = dr.models.deployment.DeploymentListFilters(
    role='OWNER',
    accuracy_health=dr.enums.DEPLOYMENT_ACCURACY_HEALTH_STATUS.FAILING
)
deployments = dr.Deployment.list(filters=filters)
deployments

### Retrieve a deployment

Retrieve a specific deployment by specifying its [deployment ID](https://docs.datarobot.com/en/docs/predictions/predapi/dep-pred.html#predictions-for-deployments), rather than listing all deployments.

In [11]:
deployment = dr.Deployment.get(deployment_id='<deployment_id>') # Provide your own deployment ID

### Update a deployment

Use the command below to update the label and description for a deployment.

In [12]:
deployment = dr.Deployment.get(deployment_id='<deployment_id>') # Provide your own deployment ID
deployment.update(label='New label')

### Delete a deployment

In [ ]:
deployment = dr.Deployment.get(deployment_id='<deployment_id>') # Provide your own deployment ID
deployment.delete()

### Model replacement

You can replace a deployment's model without interrupting predictions. Model replacement is an asynchronous process which requires prepatory steps to complete the process. However, predictions made against the deployment will use the new model as soon as you initiate the replacement process. Note that the <code>replace_model()</code> function will not return until this process is fully completed.

Alongside the new model's model ID, you must provide a reason for replacement, stored in the model history of the deployment for auditing purposes. An enum, <code>MODEL_REPLACEMENT_REASON</code>, is provided for this purpose, and all possible values are listed below:

- MODEL_REPLACEMENT_REASON.ACCURACY
- MODEL_REPLACEMENT_REASON.DATA_DRIFT
- MODEL_REPLACEMENT_REASON.ERRORS
- MODEL_REPLACEMENT_REASON.SCHEDULED_REFRESH
- MODEL_REPLACEMENT_REASON.SCORING_SPEED
- MODEL_REPLACEMENT_REASON.OTHER

The code below demonstrates an example of model replacement:

In [ ]:
from datarobot.enums import MODEL_REPLACEMENT_REASON

deployment=deployment.get(deployment_id='<deployment_id>') # Provide your own deployment ID
deployment.model['id'], deployment.model['type']

deployment.replace_model('<deployment_id>', MODEL_REPLACEMENT_REASON.ACCURACY) #Provide the new model ID
deployment.model['id'], deployment.model['type']

#### Model validation (pre-replacement)

Before initiating a model replacement request, DataRobot recommends using the <code>validate_replacement_model()</code> function to verify if the new model can be used as a replacement.

The <code>validate_replacement_model()</code> function returns the validation status, a message, and a checks dictionary. If the status is `passing` or `warning`, use <code>replace_model()</code> to replace the model. If the status is <code>failing</code>, refer to the checks dictionary for information as to why the new model cannot be used as a replacement.


In [ ]:
deployment = dr.Deployment.get(deployment_id='<deployment_id>')
status, message, checks = deployment.validate_replacement_model(new_model_id=model.id)
status

# `checks` can be inspected for detail, showing two examples here:
checks['target']
checks['permission']

## Model monitoring

Deployment monitoring can be summarized by its three major components: [service health](https://docs.datarobot.com/en/docs/mlops/monitor/service-health.html), [data drift](https://docs.datarobot.com/en/docs/mlops/monitor/data-drift.html#data-drift-tab), and [accuracy](https://docs.datarobot.com/en/docs/mlops/monitor/deploy-accuracy.html). For a Deployment object, DataRobot provides `get` functions that allows you to query all of the monitoring data. Alternatively, you can retrieve monitoring data directly using a deployment ID.


In [22]:
from datarobot.models import Deployment, ServiceStats

deployment_id = '<deployment_id>'

# call `get` functions on a `Deployment` object
deployment = Deployment.get(deployment_id)
service_stats = deployment.get_service_stats()

# directly fetch without a `Deployment` object
service_stats = ServiceStats.get(deployment_id)

When querying monitoring data, you can optionally provide a start and end time. DataRobot accepts either a datetime object or a string. Note that only top of the hour datetimes are accepted, for example: `2019-08-01T00:00:00Z`. By default, the end time of the query will be the next top of the hour, and the start time will be 7 days before the end time.

In the "over time" variants, an optional `bucket_size` can be provided to specify the resolution of time buckets. For example, if the start time is `2019-08-01T00:00:00Z`, then the end time is `2019-08-02T00:00:00Z` and the `bucket_size` is `T1H`. In this case 24 time buckets are generated, each providing data calculated over one hour. Use `construct_duration_string()` to help construct a bucket size string.

### Service health

Service health metrics capture a deployment’s ability to respond to prediction requests quickly and reliably. This helps identify bottlenecks and assess capacity, which is critical to proper provisioning. Use `SERVICE_STAT_METRIC.ALL` to retrieve a list of supported metrics.

`ServiceStats` retrieves values for all service stats metrics; `ServiceStatsOverTime` can be used to fetch how one single metric changes over time.

In [ ]:
from datetime import datetime
from datarobot.enums import SERVICE_STAT_METRIC
from datarobot.helpers.partitioning_methods import construct_duration_string
from datarobot.models import Deployment

deployment = Deployment.get(deployment_id='<deployment_id>')
service_stats = deployment.get_service_stats(
    start_time=datetime(2019, 8, 1, hour=15),
    end_time=datetime(2019, 8, 8, hour=15)
)
service_stats[SERVICE_STAT_METRIC.TOTAL_PREDICTIONS]

total_predictions = deployment.get_service_stats_over_time(
    start_time=datetime(2019, 8, 1, hour=15),
    end_time=datetime(2019, 8, 8, hour=15),
    bucket_size=construct_duration_string(days=1),
    metric=SERVICE_STAT_METRIC.TOTAL_PREDICTIONS
)
total_predictions.bucket_values

### Data drift

As training and production data change over time, a deployed model loses predictive power. The data surrounding the model is said to be drifting. By leveraging the training data and prediction data (also known as inference data) that is added to your deployment, data drift helps you to analyze a model's performance after it has been deployed.

Deployment’s target drift and feature drift can be retrieved separately using `datarobot.models.TargetDrift` and `datarobot.models.FeatureDrift`. Use `DATA_DRIFT_METRIC.ALL` to retrieve a list of supported metrics.

In [ ]:
from datetime import datetime
from datarobot.enums import DATA_DRIFT_METRIC
from datarobot.models import Deployment, FeatureDrift

deployment = Deployment.get(deployment_id='<deployment_id>')
target_drift = deployment.get_target_drift(
    start_time=datetime(2019, 8, 1, hour=15),
    end_time=datetime(2019, 8, 8, hour=15)
)
feature_drift_data = FeatureDrift.list(
    deployment_id='<deployment_id>',
    start_time=datetime(2019, 8, 1, hour=15),
    end_time=datetime(2019, 8, 8, hour=15),
    metric=DATA_DRIFT_METRIC.HELLINGER
)
feature_drift = feature_drift_data[0]
feature_drift.name
feature_drift.drift_score

### Accuracy

Accuracy metrics help determine whether a model's quality is decaying and if you should consider replacing it. A collection of metrics are provided to measure the accuracy of a deployment’s predictions. For deployments with classification models, use `ACCURACY_METRIC.ALL_CLASSIFICATION` for all supported metrics. For deployments with regression models, use `ACCURACY_METRIC.ALL_REGRESSION` instead.

`Accuracy` and `AccuracyOverTime` are provided to retrieve all default accuracy metrics and how one single metric change over time.

In [ ]:
from datetime import datetime
from datarobot.enums import ACCURACY_METRIC
from datarobot.helpers.partitioning_methods import construct_duration_string
from datarobot.models import Deployment

deployment = Deployment.get(deployment_id='<deployment_id>')
accuracy = deployment.get_accuracy(
    start_time=datetime(2021, 8, 1, hour=15),
    end_time=datetime(2021, 8, 1, 15, 0)
)
accuracy[ACCURACY_METRIC.RMSE]

rmse = deployment.get_accuracy_over_time(
    start_time=datetime(2021, 12, 1),
    end_time=datetime(2021, 12, 3),
    bucket_size=construct_duration_string(days=1),
    metric=ACCURACY_METRIC.RMSE
)
rmse.bucket_values